In [207]:
import pandas as pd
import numpy as np
import rebound
from celmech import Andoyer, Poincare
from celmech.poincare import PoincareParticle
import random

In [224]:
X = pd.read_csv('/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/stability/MLstability/csvs/restransformationinput.csv', index_col=0)
X.tail()

,a10,j,k,m1,m2,M1,M2,n1,n2,a1cosl1,a1sinl1,a2cosl2,a2sinl2,k1,h1,k2,h2
206558,0.993335,6,1,1.825833e-05,2.473203e-07,1.000000,0.999982,6.283113,5.226363,-0.933111,-0.359610,-1.081222,0.330554,0.023513,-0.045518,0.006770,-0.019543
206559,1.005723,7,1,1.825833e-05,2.473203e-07,1.000000,0.999982,6.283113,5.226363,-0.933111,-0.359610,-1.081222,0.330554,0.023513,-0.045518,0.006770,-0.019543
206560,0.995496,11,2,1.825833e-05,2.473203e-07,1.000000,0.999982,6.283113,5.226363,-0.933111,-0.359610,-1.081222,0.330554,0.023513,-0.045518,0.006770,-0.019543
206561,1.135844,4,1,2.473203e-07,2.124636e-06,0.999982,0.999984,5.226363,3.835267,-1.081222,0.330554,-0.829507,-1.114980,0.006770,-0.019543,0.029805,0.003590
206562,1.133324,7,2,2.473203e-07,2.124636e-06,0.999982,0.999984,5.226363,3.835267,-1.081222,0.330554,-0.829507,-1.114980,0.006770,-0.019543,0.029805,0.003590


In [225]:
y = pd.read_csv('/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/stability/MLstability/csvs/restransformationlabels.csv', index_col=0)
y.tail()

,Zcosphi,Zsinphi,Zcomcosphicom,Zcomsinphicom,Phiprime,dKprime,costheta,sintheta,costheta1,sintheta1,alpha,res_strength,Zstar,Zstarnonres,Zstarunstable,Zsep_inner,Zsep_outer
206558,-0.017132,-0.009451,0.023369,-0.045322,1.869298,0.003360,0.887993,-0.459857,0.833868,0.551964,0.884475,0.065879,0.017602,0.002542,0.015060,0.005600,0.024520
206559,-0.019549,0.003126,0.023292,-0.045174,-8.904497,-0.002724,0.415431,-0.909625,0.877093,0.480321,0.884475,0.003413,0.000491,NaN,NaN,NaN,NaN
206560,-0.001851,0.019096,0.023359,-0.045301,77.022276,0.002199,0.957922,-0.287028,0.949548,0.313622,0.884475,0.000550,0.038590,0.000000,0.038258,0.034850,0.041999
206561,0.017192,0.016578,0.031046,0.001226,-33.128483,0.004074,0.097465,0.995239,0.288914,0.957355,0.813574,0.003680,0.000078,NaN,NaN,NaN,NaN
206562,0.022592,0.007894,0.031093,0.001157,1737.026036,-0.009002,-0.147153,0.989114,0.723004,0.690844,0.813574,0.000269,0.059340,0.000000,0.059317,0.058166,0.060491


In [226]:
df = X.join(y, how='outer')
df.columns

Index(['a10', 'j', 'k', 'm1', 'm2', 'M1', 'M2', 'n1', 'n2', 'a1cosl1',
       'a1sinl1', 'a2cosl2', 'a2sinl2', 'k1', 'h1', 'k2', 'h2', 'Zcosphi',
       'Zsinphi', 'Zcomcosphicom', 'Zcomsinphicom', 'Phiprime', 'dKprime',
       'costheta', 'sintheta', 'costheta1', 'sintheta1', 'alpha',
       'res_strength', 'Zstar', 'Zstarnonres', 'Zstarunstable', 'Zsep_inner',
       'Zsep_outer'],
      dtype='object')

In [227]:
df.tail()

,a10,j,k,m1,m2,M1,M2,n1,n2,a1cosl1,...,sintheta,costheta1,sintheta1,alpha,res_strength,Zstar,Zstarnonres,Zstarunstable,Zsep_inner,Zsep_outer
206558,0.993335,6,1,1.825833e-05,2.473203e-07,1.000000,0.999982,6.283113,5.226363,-0.933111,...,-0.459857,0.833868,0.551964,0.884475,0.065879,0.017602,0.002542,0.015060,0.005600,0.024520
206559,1.005723,7,1,1.825833e-05,2.473203e-07,1.000000,0.999982,6.283113,5.226363,-0.933111,...,-0.909625,0.877093,0.480321,0.884475,0.003413,0.000491,NaN,NaN,NaN,NaN
206560,0.995496,11,2,1.825833e-05,2.473203e-07,1.000000,0.999982,6.283113,5.226363,-0.933111,...,-0.287028,0.949548,0.313622,0.884475,0.000550,0.038590,0.000000,0.038258,0.034850,0.041999
206561,1.135844,4,1,2.473203e-07,2.124636e-06,0.999982,0.999984,5.226363,3.835267,-1.081222,...,0.995239,0.288914,0.957355,0.813574,0.003680,0.000078,NaN,NaN,NaN,NaN
206562,1.133324,7,2,2.473203e-07,2.124636e-06,0.999982,0.999984,5.226363,3.835267,-1.081222,...,0.989114,0.723004,0.690844,0.813574,0.000269,0.059340,0.000000,0.059317,0.058166,0.060491


# Scale Invariance

In [190]:
fac = 3.
ID = 0
t = df.loc[ID]

t['m1'] *= fac
t['m2'] *= fac
t['M1'] *= fac
t['M2'] *= fac
t['n1'] *= np.sqrt(fac)
t['n2'] *= np.sqrt(fac)

p1 = PoincareParticle(m=t['m1'], M=t['M1'], l=np.arctan2(t['a1sinl1'], t['a1cosl1']), gamma=-np.arctan2(t['h1'], t['k1']), a=np.sqrt(t['a1cosl1']**2 + t['a1sinl1']**2), e=np.sqrt(t['h1']**2 + t['k1']**2))
p2 = PoincareParticle(m=t['m2'], M=t['M2'], l=np.arctan2(t['a2sinl2'], t['a2cosl2']), gamma=-np.arctan2(t['h2'], t['k2']), a=np.sqrt(t['a2cosl2']**2 + t['a2sinl2']**2), e=np.sqrt(t['h2']**2 + t['k2']**2))
pvars = Poincare(1., [p1, p2])
a = Andoyer.from_Poincare(pvars, j=int(t['j']), k=int(t['k']), a10=t['a10'])

/home/dtamayo/miniconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/dtamayo/miniconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/dtamayo/miniconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/dtamayo/miniconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launche

In [192]:
print('Recalculated Returned from func Frac. Error')
print(a.Z*np.cos(a.phi), t['Zcosphi'], np.abs((a.Z*np.cos(a.phi)-t['Zcosphi'])/t['Zcosphi']))
print(a.Z*np.sin(a.phi), t['Zsinphi'], np.abs((a.Z*np.sin(a.phi)-t['Zsinphi'])/t['Zsinphi']))
print(a.Zcom*np.cos(a.phiZcom), t['Zcomcosphicom'], np.abs((a.Zcom*np.cos(a.phiZcom)-t['Zcomcosphicom'])/t['Zcomcosphicom']))
print(a.Zcom*np.sin(a.phiZcom), t['Zcomsinphicom'], np.abs((a.Zcom*np.sin(a.phiZcom)-t['Zcomsinphicom'])/t['Zcomsinphicom']))
print(a.Phiprime, t['Phiprime'], np.abs((a.Phiprime - t['Phiprime'])/t['Phiprime']))
print(a.dKprime, t['dKprime'], np.abs((a.dKprime-t['dKprime'])/t['dKprime']))
print(np.cos(a.theta), t['costheta'], np.abs((np.cos(a.theta)-t['costheta'])/t['costheta']))
print(np.sin(a.theta), t['sintheta'], np.abs((np.sin(a.theta)-t['sintheta'])/t['sintheta']))
print(np.cos(a.theta1), t['costheta1'], np.abs((np.cos(a.theta1)-t['costheta1'])/t['costheta1']))
print(np.sin(a.theta1), t['sintheta1'], np.abs((np.sin(a.theta1)-t['sintheta1'])/t['sintheta1']))
print(a.Zstar, t['Zstar'], np.abs((a.Zstar - t['Zstar'])/t['Zstar']))
print(a.Zstar_nonres, t['Zstarnonres'], np.abs((a.Zstar_nonres - t['Zstarnonres'])/t['Zstarnonres']))
print(a.Zstar_unstable, t['Zstarunstable'], np.abs((a.Zstar_unstable - t['Zstarunstable'])/t['Zstarunstable']))
print(a.Zsep_inner, t['Zsep_inner'], np.abs((a.Zsep_inner - t['Zsep_inner'])/t['Zsep_inner']))
print(a.Zsep_outer, t['Zsep_outer'], np.abs((a.Zsep_outer - t['Zsep_outer'])/t['Zsep_outer']))
resstrength = np.abs(np.sqrt(t['m1']/t['M1'] + t['m2']/t['M2'])*np.sqrt((t['Zcosphi']**2 + t['Zsinphi']**2))**(t['k']/2.)*t['n1']/(t['j']*t['n2'] - (t['j']-t['k'])*t['n1']))
print(resstrength, t['res_strength'], np.abs((resstrength - t['res_strength'])/t['res_strength']))

Recalculated Returned from func Frac. Error
-0.02160912851175698 -0.021609128511756684 1.3647148738812488e-14
-0.00028197466850127104 -0.0002819746685027645 5.29634200979836e-12
0.04106824537813678 0.04106824537813564 2.7878412715295513e-14
-0.006615110637094021 -0.006615110637093807 3.238620803736946e-14
6.746062988941958 6.746062988941735 3.3046353659929765e-14
-0.0034243539600915317 -0.0034243539600915807 1.4311002532878988e-14
0.41846539773251296 0.41846539773251296 0.0
-0.9082327404914281 -0.908232740491428 1.2223992542092627e-16
0.9907173288601742 0.9907173288601742 0.0
0.13593812672006864 0.13593812672006866 2.0417800572450683e-16
0.021608685074271435 0.0216086850742711 1.557412462548245e-14
0.00046983151630428945 0.0004698315163043037 3.034547081969304e-14
0.021138853557967147 0.021138853557966783 1.7233286987686977e-14
0.016632759623352186 0.016632759623351732 2.7325444544261135e-14
0.025644947492582115 0.02564494749258184 1.0687731346832307e-14
0.13721373447434268 0.137213734

In [193]:
def rescale(row, augment_factor=5):
    rows = []
    for i in range(augment_factor):
        t = row.copy()
        mlogscale = random.uniform(-1, 1)
        mfac = 10**mlogscale
        alogscale = random.uniform(-2, 2)
        afac = 10**alogscale
        t['m1'] *= mfac
        t['m2'] *= mfac
        t['M1'] *= mfac
        t['M2'] *= mfac
        t['a10'] *= afac
        t['a1cosl1'] *= afac
        t['a1sinl1'] *= afac
        t['a2cosl2'] *= afac
        t['a2sinl2'] *= afac
        t['n1'] *= np.sqrt(mfac)/afac**1.5
        t['n2'] *= np.sqrt(mfac)/afac**1.5
        
        rows.append(t)
        
    return pd.DataFrame(data=rows)

# Test that transformed inputs give same outputs

In [194]:
test = rescale(df.loc[0])
test

,a10,j,k,m1,m2,M1,M2,n1,n2,a1cosl1,...,sintheta,costheta1,sintheta1,alpha,res_strength,Zstar,Zstarnonres,Zstarunstable,Zsep_inner,Zsep_outer
0,20.674956,12.0,1.0,0.000008,1.634108e-07,0.645464,0.645456,0.054253,0.049714,-0.943375,...,-0.908233,0.990717,0.135938,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645
0,1.220782,12.0,1.0,0.000005,1.023383e-07,0.404231,0.404226,2.992329,2.742017,-0.055703,...,-0.908233,0.990717,0.135938,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645
0,64.968341,12.0,1.0,0.000018,3.716864e-07,1.468141,1.468123,0.014689,0.013460,-2.964431,...,-0.908233,0.990717,0.135938,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645
0,0.203049,12.0,1.0,0.000013,2.586939e-07,1.021827,1.021815,70.135544,64.268603,-0.009265,...,-0.908233,0.990717,0.135938,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645
0,4.505326,12.0,1.0,0.000040,8.222823e-07,3.247971,3.247931,1.196379,1.096300,-0.205573,...,-0.908233,0.990717,0.135938,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645


In [199]:
for i, t in test.iterrows():
    p1 = PoincareParticle(m=t['m1'], M=t['M1'], l=np.arctan2(t['a1sinl1'], t['a1cosl1']), gamma=-np.arctan2(t['h1'], t['k1']), a=np.sqrt(t['a1cosl1']**2 + t['a1sinl1']**2), e=np.sqrt(t['h1']**2 + t['k1']**2))
    p2 = PoincareParticle(m=t['m2'], M=t['M2'], l=np.arctan2(t['a2sinl2'], t['a2cosl2']), gamma=-np.arctan2(t['h2'], t['k2']), a=np.sqrt(t['a2cosl2']**2 + t['a2sinl2']**2), e=np.sqrt(t['h2']**2 + t['k2']**2))
    pvars = Poincare(1., [p1, p2])
    a = Andoyer.from_Poincare(pvars, j=int(t['j']), k=int(t['k']), a10=t['a10'])

    print('Recalculated Returned from func Frac. Error')
    print('***********')
    print(a.Z*np.cos(a.phi), t['Zcosphi'], np.abs((a.Z*np.cos(a.phi)-t['Zcosphi'])/t['Zcosphi']))
    print(a.Z*np.sin(a.phi), t['Zsinphi'], np.abs((a.Z*np.sin(a.phi)-t['Zsinphi'])/t['Zsinphi']))
    print(a.Zcom*np.cos(a.phiZcom), t['Zcomcosphicom'], np.abs((a.Zcom*np.cos(a.phiZcom)-t['Zcomcosphicom'])/t['Zcomcosphicom']))
    print(a.Zcom*np.sin(a.phiZcom), t['Zcomsinphicom'], np.abs((a.Zcom*np.sin(a.phiZcom)-t['Zcomsinphicom'])/t['Zcomsinphicom']))
    print(a.Phiprime, t['Phiprime'], np.abs((a.Phiprime - t['Phiprime'])/t['Phiprime']))
    print(a.dKprime, t['dKprime'], np.abs((a.dKprime-t['dKprime'])/t['dKprime']))
    print(np.cos(a.theta), t['costheta'], np.abs((np.cos(a.theta)-t['costheta'])/t['costheta']))
    print(np.sin(a.theta), t['sintheta'], np.abs((np.sin(a.theta)-t['sintheta'])/t['sintheta']))
    print(np.cos(a.theta1), t['costheta1'], np.abs((np.cos(a.theta1)-t['costheta1'])/t['costheta1']))
    print(np.sin(a.theta1), t['sintheta1'], np.abs((np.sin(a.theta1)-t['sintheta1'])/t['sintheta1']))
    print(a.Zstar, t['Zstar'], np.abs((a.Zstar - t['Zstar'])/t['Zstar']))
    print(a.Zstar_nonres, t['Zstarnonres'], np.abs((a.Zstar_nonres - t['Zstarnonres'])/t['Zstarnonres']))
    print(a.Zstar_unstable, t['Zstarunstable'], np.abs((a.Zstar_unstable - t['Zstarunstable'])/t['Zstarunstable']))
    print(a.Zsep_inner, t['Zsep_inner'], np.abs((a.Zsep_inner - t['Zsep_inner'])/t['Zsep_inner']))
    print(a.Zsep_outer, t['Zsep_outer'], np.abs((a.Zsep_outer - t['Zsep_outer'])/t['Zsep_outer']))
    resstrength = np.abs(np.sqrt(t['m1']/t['M1'] + t['m2']/t['M2'])*np.sqrt((t['Zcosphi']**2 + t['Zsinphi']**2))**(t['k']/2.)*t['n1']/(t['j']*t['n2'] - (t['j']-t['k'])*t['n1']))
    print(resstrength, t['res_strength'], np.abs((resstrength - t['res_strength'])/t['res_strength']))
    print('***********')

Recalculated Returned from func Frac. Error
***********
-0.021609128511757 -0.021609128511756684 1.4610476885081605e-14
-0.00028197466850134813 -0.0002819746685027645 5.022960096192304e-12
0.017475537646037858 0.017475537646037403 2.6007643364779752e-14
-0.03774872251032823 -0.037748722510327154 2.849178683626661e-14
6.746062988941618 6.746062988941735 1.7378958896855495e-14
-0.0034243539600916054 -0.0034243539600915807 7.21882428649648e-15
-0.5190775241827532 -0.5190775241827545 2.5666062726328505e-15
-0.8547271634201777 -0.854727163420177 9.092446695245205e-16
0.9985010295821467 0.9985010295821467 0.0
0.05473293271324763 0.05473293271324765 3.8033192595000814e-16
0.02160868507427091 0.0216086850742711 8.670131234804662e-15
0.0004698315163042995 0.0004698315163043037 8.999797429414666e-15
0.021138853557966592 0.021138853557966783 9.026959850693177e-15
0.01663275962335152 0.016632759623351732 1.2724061963358238e-14
0.02564494749258166 0.02564494749258184 7.034962405509873e-15
0.1372137

In [196]:
def rotate(row, augment_factor=5):
    rows = []
    for i in range(augment_factor):
        t = row.copy()
        
        rot = random.uniform(0., 2*np.pi)
        pomega1 = np.arctan2(t['h1'], t['k1']) + rot
        pomega2 = np.arctan2(t['h2'], t['k2']) + rot
        l1 = np.arctan2(t['a1sinl1'], t['a1cosl1']) + rot
        l2 = np.arctan2(t['a2sinl2'], t['a2cosl2']) + rot
        
        a1 = np.sqrt(t['a1cosl1']**2 + t['a1sinl1']**2)
        a2 = np.sqrt(t['a2cosl2']**2 + t['a2sinl2']**2)
        e1 = np.sqrt(t['h1']**2 + t['k1']**2)
        e2 = np.sqrt(t['h2']**2 + t['k2']**2)
        
        t['a1cosl1'] = a1*np.cos(l1)
        t['a1sinl1'] = a1*np.sin(l1)
        t['a2cosl2'] = a2*np.cos(l2)
        t['a2sinl2'] = a2*np.sin(l2)
        t['k1'] = e1*np.cos(pomega1)
        t['h1'] = e1*np.sin(pomega1)
        t['k2'] = e2*np.cos(pomega2)
        t['h2'] = e2*np.sin(pomega2)
        
        phiZcom = np.arctan2(t['Zcomsinphicom'], t['Zcomcosphicom']) + rot
        theta = np.arctan2(t['sintheta'], t['costheta']) + t['k']*rot
        
        p1 = PoincareParticle(m=t['m1'], M=t['M1'], l=np.arctan2(t['a1sinl1'], t['a1cosl1']), gamma=-np.arctan2(t['h1'], t['k1']), a=np.sqrt(t['a1cosl1']**2 + t['a1sinl1']**2), e=np.sqrt(t['h1']**2 + t['k1']**2))
        p2 = PoincareParticle(m=t['m2'], M=t['M2'], l=np.arctan2(t['a2sinl2'], t['a2cosl2']), gamma=-np.arctan2(t['h2'], t['k2']), a=np.sqrt(t['a2cosl2']**2 + t['a2sinl2']**2), e=np.sqrt(t['h2']**2 + t['k2']**2))
        pvars = Poincare(1., [p1, p2])
        a = Andoyer.from_Poincare(pvars, j=int(t['j']), k=int(t['k']), a10=t['a10'])
        p = a.params
        #theta1 = np.arctan2(t['dKprimesintheta1'], t['dKprimecostheta1'])
        theta1 = (p['m1']*p['sLambda10']*np.mod(l1, 2*np.pi) + p['m2']*p['sLambda20']*np.mod(l2, 2*np.pi))/p['K0']

        Zcom = np.sqrt(t['Zcomcosphicom']**2 + t['Zcomsinphicom']**2)
        
        t['Zcomcosphicom'] = Zcom*np.cos(phiZcom)
        t['Zcomsinphicom'] = Zcom*np.sin(phiZcom)
        t['costheta'] = np.cos(theta)
        t['sintheta'] = np.sin(theta)
        t['costheta1'] = np.cos(theta1)
        t['sintheta1'] = np.sin(theta1)
        
        rows.append(t)
        
    return pd.DataFrame(data=rows)

In [203]:
random.seed(0)
test = rotate(df.loc[0])
test

,a10,j,k,m1,m2,M1,M2,n1,n2,a1cosl1,...,sintheta,costheta1,sintheta1,alpha,res_strength,Zstar,Zstarnonres,Zstarunstable,Zsep_inner,Zsep_outer
0,1.514434,12.0,1.0,0.000012,2.531702e-07,1.000008,0.999996,3.406265,3.121326,1.207102,...,-0.854727,0.998501,0.054733,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645
0,1.514434,12.0,1.0,0.000012,2.531702e-07,1.000008,0.999996,3.406265,3.121326,1.497156,...,-0.463317,0.999956,0.009408,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645
0,1.514434,12.0,1.0,0.000012,2.531702e-07,1.000008,0.999996,3.406265,3.121326,-0.658421,...,0.997736,0.936968,0.349416,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645
0,1.514434,12.0,1.0,0.000012,2.531702e-07,1.000008,0.999996,3.406265,3.121326,-1.496258,...,0.468666,0.963192,0.268813,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645
0,1.514434,12.0,1.0,0.000012,2.531702e-07,1.000008,0.999996,3.406265,3.121326,0.175277,...,0.876335,0.919291,0.393579,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645


In [204]:
for i, t in test.iterrows():
    p1 = PoincareParticle(m=t['m1'], M=t['M1'], l=np.arctan2(t['a1sinl1'], t['a1cosl1']), gamma=-np.arctan2(t['h1'], t['k1']), a=np.sqrt(t['a1cosl1']**2 + t['a1sinl1']**2), e=np.sqrt(t['h1']**2 + t['k1']**2))
    p2 = PoincareParticle(m=t['m2'], M=t['M2'], l=np.arctan2(t['a2sinl2'], t['a2cosl2']), gamma=-np.arctan2(t['h2'], t['k2']), a=np.sqrt(t['a2cosl2']**2 + t['a2sinl2']**2), e=np.sqrt(t['h2']**2 + t['k2']**2))
    pvars = Poincare(1., [p1, p2])
    a = Andoyer.from_Poincare(pvars, j=int(t['j']), k=int(t['k']), a10=t['a10'])

    print('Recalculated Returned from func Frac. Error')
    print('***********')
    print(a.Z*np.cos(a.phi), t['Zcosphi'], np.abs((a.Z*np.cos(a.phi)-t['Zcosphi'])/t['Zcosphi']))
    print(a.Z*np.sin(a.phi), t['Zsinphi'], np.abs((a.Z*np.sin(a.phi)-t['Zsinphi'])/t['Zsinphi']))
    print(a.Zcom*np.cos(a.phiZcom), t['Zcomcosphicom'], np.abs((a.Zcom*np.cos(a.phiZcom)-t['Zcomcosphicom'])/t['Zcomcosphicom']))
    print(a.Zcom*np.sin(a.phiZcom), t['Zcomsinphicom'], np.abs((a.Zcom*np.sin(a.phiZcom)-t['Zcomsinphicom'])/t['Zcomsinphicom']))
    print(a.Phiprime, t['Phiprime'], np.abs((a.Phiprime - t['Phiprime'])/t['Phiprime']))
    print(a.dKprime, t['dKprime'], np.abs((a.dKprime-t['dKprime'])/t['dKprime']))
    print(np.cos(a.theta), t['costheta'], np.abs((np.cos(a.theta)-t['costheta'])/t['costheta']))
    print(np.sin(a.theta), t['sintheta'], np.abs((np.sin(a.theta)-t['sintheta'])/t['sintheta']))
    print(np.cos(a.theta1), t['costheta1'], np.abs((np.cos(a.theta1)-t['costheta1'])/t['costheta1']))
    print(np.sin(a.theta1), t['sintheta1'], np.abs((np.sin(a.theta1)-t['sintheta1'])/t['sintheta1']))
    print(a.Zstar, t['Zstar'], np.abs((a.Zstar - t['Zstar'])/t['Zstar']))
    print(a.Zstar_nonres, t['Zstarnonres'], np.abs((a.Zstar_nonres - t['Zstarnonres'])/t['Zstarnonres']))
    print(a.Zstar_unstable, t['Zstarunstable'], np.abs((a.Zstar_unstable - t['Zstarunstable'])/t['Zstarunstable']))
    print(a.Zsep_inner, t['Zsep_inner'], np.abs((a.Zsep_inner - t['Zsep_inner'])/t['Zsep_inner']))
    print(a.Zsep_outer, t['Zsep_outer'], np.abs((a.Zsep_outer - t['Zsep_outer'])/t['Zsep_outer']))
    resstrength = np.abs(np.sqrt(t['m1']/t['M1'] + t['m2']/t['M2'])*np.sqrt((t['Zcosphi']**2 + t['Zsinphi']**2))**(t['k']/2.)*t['n1']/(t['j']*t['n2'] - (t['j']-t['k'])*t['n1']))
    print(resstrength, t['res_strength'], np.abs((resstrength - t['res_strength'])/t['res_strength']))
    print('***********')

Recalculated Returned from func Frac. Error
***********
-0.021609128511757 -0.021609128511756684 1.4610476885081605e-14
-0.00028197466850134813 -0.0002819746685027645 5.022960096192304e-12
0.017475537646037858 0.017475537646037403 2.6007643364779752e-14
-0.03774872251032823 -0.037748722510327154 2.849178683626661e-14
6.746062988941618 6.746062988941735 1.7378958896855495e-14
-0.0034243539600916054 -0.0034243539600915807 7.21882428649648e-15
-0.5190775241827532 -0.5190775241827545 2.5666062726328505e-15
-0.8547271634201777 -0.854727163420177 9.092446695245205e-16
0.9985010295821467 0.9985010295821467 0.0
0.05473293271324763 0.05473293271324765 3.8033192595000814e-16
0.02160868507427091 0.0216086850742711 8.670131234804662e-15
0.0004698315163042995 0.0004698315163043037 8.999797429414666e-15
0.021138853557966592 0.021138853557966783 9.026959850693177e-15
0.01663275962335152 0.016632759623351732 1.2724061963358238e-14
0.02564494749258166 0.02564494749258184 7.034962405509873e-15
0.1372137

In [229]:
def scale_and_rotate(row, augment_factor=5):
    rows = [row]
    for i in range(augment_factor):
        t = row.copy()
        mlogscale = random.uniform(-1, 1)
        mfac = 10**mlogscale
        alogscale = random.uniform(-2, 2)
        afac = 10**alogscale
        t['m1'] *= mfac
        t['m2'] *= mfac
        t['M1'] *= mfac
        t['M2'] *= mfac
        t['a10'] *= afac
        t['a1cosl1'] *= afac
        t['a1sinl1'] *= afac
        t['a2cosl2'] *= afac
        t['a2sinl2'] *= afac
        t['n1'] *= np.sqrt(mfac)/afac**1.5
        t['n2'] *= np.sqrt(mfac)/afac**1.5
        
        rot = random.uniform(0., 2*np.pi)
        pomega1 = np.arctan2(t['h1'], t['k1']) + rot
        pomega2 = np.arctan2(t['h2'], t['k2']) + rot
        l1 = np.arctan2(t['a1sinl1'], t['a1cosl1']) + rot
        l2 = np.arctan2(t['a2sinl2'], t['a2cosl2']) + rot
        
        a1 = np.sqrt(t['a1cosl1']**2 + t['a1sinl1']**2)
        a2 = np.sqrt(t['a2cosl2']**2 + t['a2sinl2']**2)
        e1 = np.sqrt(t['h1']**2 + t['k1']**2)
        e2 = np.sqrt(t['h2']**2 + t['k2']**2)
        
        t['a1cosl1'] = a1*np.cos(l1)
        t['a1sinl1'] = a1*np.sin(l1)
        t['a2cosl2'] = a2*np.cos(l2)
        t['a2sinl2'] = a2*np.sin(l2)
        t['k1'] = e1*np.cos(pomega1)
        t['h1'] = e1*np.sin(pomega1)
        t['k2'] = e2*np.cos(pomega2)
        t['h2'] = e2*np.sin(pomega2)
        
        phiZcom = np.arctan2(t['Zcomsinphicom'], t['Zcomcosphicom']) + rot
        theta = np.arctan2(t['sintheta'], t['costheta']) + t['k']*rot
        
        p1 = PoincareParticle(m=t['m1'], M=t['M1'], l=np.arctan2(t['a1sinl1'], t['a1cosl1']), gamma=-np.arctan2(t['h1'], t['k1']), a=np.sqrt(t['a1cosl1']**2 + t['a1sinl1']**2), e=np.sqrt(t['h1']**2 + t['k1']**2))
        p2 = PoincareParticle(m=t['m2'], M=t['M2'], l=np.arctan2(t['a2sinl2'], t['a2cosl2']), gamma=-np.arctan2(t['h2'], t['k2']), a=np.sqrt(t['a2cosl2']**2 + t['a2sinl2']**2), e=np.sqrt(t['h2']**2 + t['k2']**2))
        pvars = Poincare(1., [p1, p2])
        a = Andoyer.from_Poincare(pvars, j=int(t['j']), k=int(t['k']), a10=t['a10'])
        p = a.params
        #theta1 = np.arctan2(t['dKprimesintheta1'], t['dKprimecostheta1'])
        theta1 = (p['m1']*p['sLambda10']*np.mod(l1, 2*np.pi) + p['m2']*p['sLambda20']*np.mod(l2, 2*np.pi))/p['K0']

        Zcom = np.sqrt(t['Zcomcosphicom']**2 + t['Zcomsinphicom']**2)
        
        t['Zcomcosphicom'] = Zcom*np.cos(phiZcom)
        t['Zcomsinphicom'] = Zcom*np.sin(phiZcom)
        t['costheta'] = np.cos(theta)
        t['sintheta'] = np.sin(theta)
        t['costheta1'] = np.cos(theta1)
        t['sintheta1'] = np.sin(theta1)
        
        rows.append(t)
        
    return pd.DataFrame(data=rows)

In [205]:
random.seed(0)
test = scale_and_rotate(df.loc[0])
test

,a10,j,k,m1,m2,M1,M2,n1,n2,a1cosl1,...,sintheta,costheta1,sintheta1,alpha,res_strength,Zstar,Zstarnonres,Zstarunstable,Zsep_inner,Zsep_outer
0,16.295506,12.0,1.0,0.000061,1.236677e-06,4.884807,4.884747,0.213292,0.195449,-7.084694,...,0.997736,0.936968,0.349416,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645
0,1.680154,12.0,1.0,0.000004,8.341536e-08,0.329486,0.329482,1.673196,1.533231,-0.874117,...,0.986326,0.939802,0.341720,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645
0,0.247458,12.0,1.0,0.000046,9.354266e-07,3.694884,3.694839,99.128974,90.836690,-0.024801,...,0.959743,0.926297,0.376794,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645
0,64.968341,12.0,1.0,0.000018,3.716864e-07,1.468141,1.468123,0.014689,0.013460,4.860989,...,0.895518,0.920645,0.390400,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645
0,15.975963,12.0,1.0,0.000005,9.270181e-08,0.366167,0.366163,0.060158,0.055125,11.267517,...,0.385092,0.900476,0.434905,0.943428,0.137214,0.021609,0.00047,0.021139,0.016633,0.025645


In [206]:
for i, t in test.iterrows():
    p1 = PoincareParticle(m=t['m1'], M=t['M1'], l=np.arctan2(t['a1sinl1'], t['a1cosl1']), gamma=-np.arctan2(t['h1'], t['k1']), a=np.sqrt(t['a1cosl1']**2 + t['a1sinl1']**2), e=np.sqrt(t['h1']**2 + t['k1']**2))
    p2 = PoincareParticle(m=t['m2'], M=t['M2'], l=np.arctan2(t['a2sinl2'], t['a2cosl2']), gamma=-np.arctan2(t['h2'], t['k2']), a=np.sqrt(t['a2cosl2']**2 + t['a2sinl2']**2), e=np.sqrt(t['h2']**2 + t['k2']**2))
    pvars = Poincare(1., [p1, p2])
    a = Andoyer.from_Poincare(pvars, j=int(t['j']), k=int(t['k']), a10=t['a10'])

    print('Recalculated Returned from func Frac. Error')
    print('***********')
    print(a.Z*np.cos(a.phi), t['Zcosphi'], np.abs((a.Z*np.cos(a.phi)-t['Zcosphi'])/t['Zcosphi']))
    print(a.Z*np.sin(a.phi), t['Zsinphi'], np.abs((a.Z*np.sin(a.phi)-t['Zsinphi'])/t['Zsinphi']))
    print(a.Zcom*np.cos(a.phiZcom), t['Zcomcosphicom'], np.abs((a.Zcom*np.cos(a.phiZcom)-t['Zcomcosphicom'])/t['Zcomcosphicom']))
    print(a.Zcom*np.sin(a.phiZcom), t['Zcomsinphicom'], np.abs((a.Zcom*np.sin(a.phiZcom)-t['Zcomsinphicom'])/t['Zcomsinphicom']))
    print(a.Phiprime, t['Phiprime'], np.abs((a.Phiprime - t['Phiprime'])/t['Phiprime']))
    print(a.dKprime, t['dKprime'], np.abs((a.dKprime-t['dKprime'])/t['dKprime']))
    print(np.cos(a.theta), t['costheta'], np.abs((np.cos(a.theta)-t['costheta'])/t['costheta']))
    print(np.sin(a.theta), t['sintheta'], np.abs((np.sin(a.theta)-t['sintheta'])/t['sintheta']))
    print(np.cos(a.theta1), t['costheta1'], np.abs((np.cos(a.theta1)-t['costheta1'])/t['costheta1']))
    print(np.sin(a.theta1), t['sintheta1'], np.abs((np.sin(a.theta1)-t['sintheta1'])/t['sintheta1']))
    print(a.Zstar, t['Zstar'], np.abs((a.Zstar - t['Zstar'])/t['Zstar']))
    print(a.Zstar_nonres, t['Zstarnonres'], np.abs((a.Zstar_nonres - t['Zstarnonres'])/t['Zstarnonres']))
    print(a.Zstar_unstable, t['Zstarunstable'], np.abs((a.Zstar_unstable - t['Zstarunstable'])/t['Zstarunstable']))
    print(a.Zsep_inner, t['Zsep_inner'], np.abs((a.Zsep_inner - t['Zsep_inner'])/t['Zsep_inner']))
    print(a.Zsep_outer, t['Zsep_outer'], np.abs((a.Zsep_outer - t['Zsep_outer'])/t['Zsep_outer']))
    resstrength = np.abs(np.sqrt(t['m1']/t['M1'] + t['m2']/t['M2'])*np.sqrt((t['Zcosphi']**2 + t['Zsinphi']**2))**(t['k']/2.)*t['n1']/(t['j']*t['n2'] - (t['j']-t['k'])*t['n1']))
    print(resstrength, t['res_strength'], np.abs((resstrength - t['res_strength'])/t['res_strength']))
    print('***********')

Recalculated Returned from func Frac. Error
***********
-0.021609128511756975 -0.021609128511756684 1.3486594047767635e-14
-0.00028197466850133826 -0.0002819746685027645 5.057949905106721e-12
-0.03289318477000919 -0.03289318477000828 2.7634754973336022e-14
0.02546367732409999 0.02546367732409924 2.9430177428172785e-14
6.746062988942245 6.746062988941735 7.557213944541707e-14
-0.0034243539600913274 -0.0034243539600915807 7.396128742656043e-14
0.06725766153826908 0.06725766153827213 4.53942829395257e-14
0.9977356398186865 0.9977356398186863 2.2254853496601805e-16
0.9369675443925461 0.9369675443925461 0.0
0.34941640023759907 0.34941640023759907 0.0
0.02160868507427189 0.0216086850742711 3.6607220769175244e-14
0.0004698315163042776 0.0004698315163043037 5.56141328330496e-14
0.021138853557967616 0.021138853557966783 3.939037025757023e-14
0.01663275962335267 0.016632759623351732 5.631961852633974e-14
0.025644947492582556 0.02564494749258184 2.7869274144904497e-14
0.13721373447440263 0.137213

In [ ]:
%%time
augment_factor=10
augdata = pd.concat([scale_and_rotate(row, augment_factor) for _, row in df.iterrows()])

In [231]:
augdata.shape

(2272193, 34)

In [232]:
augdata = augdata[~augdata['alpha'].isnull()]
augdata.shape

(2272193, 34)

In [233]:
augdata = augdata.reset_index(drop=True)

In [234]:
augdata.to_csv('/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/stability/MLstability/csvs/restransformationAug.csv')

In [235]:
X = augdata[['a10', 'j', 'k', 'm1', 'm2', 'M1', 'M2', 'n1', 'n2', 'a1cosl1', 'a1sinl1', 'a2cosl2', 'a2sinl2', 'k1', 'h1', 'k2', 'h2']]
y = augdata[['Zcosphi', 'Zsinphi', 'Zcomcosphicom', 'Zcomsinphicom', 'Phiprime', 'dKprime', 'costheta', 'sintheta', 'costheta1', 'sintheta1', 'alpha', 'res_strength', 'Zstar', 'Zstarnonres', 'Zstarunstable', 'Zsep_inner', 'Zsep_outer']]

In [236]:
X.shape

(2272193, 17)

In [237]:
y.shape

(2272193, 17)

In [238]:
X.to_csv('/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/stability/MLstability/csvs/restransformationinputAug.csv')
y.to_csv('/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/stability/MLstability/csvs/restransformationlabelsAug.csv')